In [9]:
import urllib
import datetime
import requests
import os
import sys
import glob
import re 

from PyPDF2 import PdfFileReader,PdfFileWriter
from bs4 import BeautifulSoup  
from datetime import timedelta, date

def getFileName(filepath):
    file_list = []
    for root,dirs,files in os.walk(filepath):
        for filespath in files:
            # print(os.path.join(root,filespath))
            file_list.append(os.path.join(root,filespath))
    return file_list

def merge_separate_pdf_to_one(year, month, day):
    filepath = os.getcwd()
    current_file_name_list = getFileName(filepath)
    current_file_name_filter = "81_paper_" + year + month + day
    infnList = []

    # fliter files with current_file_name_filter
    for each_file in current_file_name_list:
        each_file_name = each_file.split('/')[-1]
        if each_file.split('/')[-1].startswith(current_file_name_filter):
            infnList.append(each_file_name)

    infnList =  sorted(infnList)
    print infnList

    pdf_output = PdfFileWriter()
    for infn in infnList:
        print infn
        pdf_input = PdfFileReader(open(infn, 'rb'), strict=False)
        # 获取 pdf 共用多少页
        page_count = pdf_input.getNumPages()
        print(page_count)
        for i in range(page_count):
            pdf_output.addPage(pdf_input.getPage(i))

    summaryName = select_day +'_81_Summary.pdf'
    pdf_output.write(open(summaryName, 'wb'))
    

def download_separate_pdf(year, month, day):
    url = "http://www.81.cn/jfjbmap/content/2018-04/15/node_2.htm"

    # http://www.81.cn/jfjbmap/content/1/2018-04/15/01/2018041501_pdf.pdf
    # http://www.81.cn/jfjbmap/content/1/2018-04/15/01/2018041501_pdf.pdf

    # http://www.81.cn/jfjbmap/content/1/2018-04/15/02/2018041502_pdf.pdf

    paper_81_base = "http://www.81.cn/jfjbmap/content/"
    paper_81_download_base = "http://www.81.cn/jfjbmap/content/1/"

    current_time = year + '-' + month + '/' + day + "/node_2.htm"
    url = paper_81_base + current_time
    html = requests.get(url) 
    html.encoding = 'utf-8' #这一行是将编码转为utf-8否则中文会显示乱码。 

    #print html.text
    soup = BeautifulSoup(html.text, "html.parser")
    a_href_list = soup.select('a[href^="node"]') # get with `a href` with `node'
    #print a_href_list
    #print len(a_href_list)

    download_pdf_list_remove_duplicate = []
    for i in a_href_list:
      if i not in download_pdf_list_remove_duplicate:
        download_pdf_list_remove_duplicate.append(i)

    print "The total number of page is " + str(len(download_pdf_list_remove_duplicate))


    download_url = paper_81_download_base + year + '-' + month + '/' + day + '/'
    print download_url
    for index  in range(0, len(download_pdf_list_remove_duplicate)):
        print index
        current_download_url = download_url + str(index+1).zfill(2) + '/' + year + month + day + str(index+1).zfill(2) + "_pdf.pdf"
        print current_download_url
        print "Downloading Page " + str(index) + " now."
        response = urllib.urlopen(current_download_url)
        current_file_name = "81_paper_" + year + month + day + str(index+1).zfill(2) + "_pdf.pdf"
        file = open(current_file_name, 'wb')
        file.write(response.read())
        file.close()
        print "Finished."
        print "---------------"

def date_range(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)
        



start_date = date(2018, 4, 29)
end_date = date(2018, 4, 30)
end_date = end_date  + datetime.timedelta(days=1)

number_of_days = abs(end_date - start_date).days
iterator_of_days = 0
#select_day = "2018.04.29"
#[year, month, day] = select_day.split(".")
for single_date in date_range(start_date, end_date):
    iterator_of_days = iterator_of_days + 1
    print single_date.strftime("%Y-%m-%d")
    t = single_date.strftime("%Y-%m-%d")
    [year, month, date] = t.split("-")
    #print year, month, date
    download_separate_pdf(year, month, day)  
    merge_separate_pdf_to_one(year, month, day)
    print"Completed Download " + year + "-"+ month + "-" + day
    print"=====================" + str(iterator_of_days) +"/" + str(number_of_days) + "==============================="
#download_separate_pdf(year, month, day)    
#merge_separate_pdf_to_one(year, month, day)    


print "Competed Download All!"
# http://www.81.cn/jfjbmap/content/1/2018-04/15/02/2018041502_pdf.pdf


2018-04-29
The total number of page is 4
http://www.81.cn/jfjbmap/content/1/2018-04/29/
0
http://www.81.cn/jfjbmap/content/1/2018-04/29/01/2018042901_pdf.pdf
Finished.
---------------
1
http://www.81.cn/jfjbmap/content/1/2018-04/29/02/2018042902_pdf.pdf
Finished.
---------------
2
http://www.81.cn/jfjbmap/content/1/2018-04/29/03/2018042903_pdf.pdf
Finished.
---------------
3
http://www.81.cn/jfjbmap/content/1/2018-04/29/04/2018042904_pdf.pdf
Finished.
---------------
['81_paper_2018042901_pdf.pdf', '81_paper_2018042902_pdf.pdf', '81_paper_2018042903_pdf.pdf', '81_paper_2018042904_pdf.pdf']
81_paper_2018042901_pdf.pdf
1
81_paper_2018042902_pdf.pdf
1
81_paper_2018042903_pdf.pdf
1
81_paper_2018042904_pdf.pdf
1
Completed Download 2018-04-29
=====================1/2===============================
2018-04-30
The total number of page is 4
http://www.81.cn/jfjbmap/content/1/2018-04/29/
0
http://www.81.cn/jfjbmap/content/1/2018-04/29/01/2018042901_pdf.pdf
Finished.
---------------
1
http://www.